In [1]:
import findspark
findspark.init()

In [2]:

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
# when the code actualy connects to hue hdfs
data_file = "/user/cloudera/Default2.csv"
df = spark.read.load(data_file, format="csv", sep=",", inferSchema="true", header="true")

In [4]:
df.show(15) # showing 15 records
print((df.count(), len(df.columns))) # 10000 rows and 5 columns

+--------+-------+-------+-----------+-----------+
|RecordID|default|student|    balance|     income|
+--------+-------+-------+-----------+-----------+
|    1137|    Yes|     No|    2499.02|51504.29396|
|       2|     No|    Yes|817.1804066| 12106.1347|
|    6076|    Yes|     PT|    2413.32|38540.57271|
|    9874|     No|     No|    2391.01|50302.90956|
|       5|     No|     PT|785.6558829|38463.49588|
|       6|     No|    Yes|919.5885305|7491.558572|
|    6335|    Yes|     PT|     2343.8|51095.29393|
|       8|     No|    Yes|808.6675043|17600.45134|
|    9894|    Yes|     PT|    2288.41|52043.56905|
|    8265|    Yes|     PT|    2236.76|37113.88307|
|      11|     No|    Yes|        0.0|21871.07309|
|      12|     No|    Yes|1220.583753|13268.56222|
|    3190|    Yes|     PT|    2228.47|27438.34899|
|    1360|    Yes|     PT|    2220.97|40725.09621|
|    9979|    Yes|     PT|    2202.46|47287.25711|
+--------+-------+-------+-----------+-----------+
only showing top 15 rows

(1000

In [5]:
# Show me all of the null rows 
# looks like 71 appears twice so when i do list removal it should be 9988 
#10000  - 12 = 9988
df.filter(df.RecordID.isNull()).show()
df.filter(df.student.isNull()).show()
df.filter(df.default.isNull()).show()
df.filter(df.income.isNull()).show()
df.filter(df.balance.isNull()).show()

+--------+-------+-------+-------+------+
|RecordID|default|student|balance|income|
+--------+-------+-------+-------+------+
+--------+-------+-------+-------+------+

+--------+-------+-------+-----------+-----------+
|RecordID|default|student|    balance|     income|
+--------+-------+-------+-----------+-----------+
|      40|     No|   null|653.1184001| 39489.5947|
|      71|     No|   null|       null|31577.61513|
+--------+-------+-------+-----------+-----------+

+--------+-------+-------+----------+-----------+
|RecordID|default|student|   balance|     income|
+--------+-------+-------+----------+-----------+
|      76|   null|    Yes|752.459946|16211.27579|
|      48|   null|     No|   1228.31|37408.50387|
|      49|   null|     No|    820.92|47746.54207|
+--------+-------+-------+----------+-----------+

+--------+-------+-------+-----------+------+
|RecordID|default|student|    balance|income|
+--------+-------+-------+-----------+------+
|      35|     No|    Yes|1423.9389

In [21]:
# Perform listwise removal on rows with empty value(s).
# Show the first 15 records to confirm the null values are gone.
df2 = df.na.drop()
df2.show(15)
print((df2.count(), len(df2.columns))) # 9988 rows and 5 columns

+--------+-------+-------+-----------+-----------+
|RecordID|default|student|    balance|     income|
+--------+-------+-------+-----------+-----------+
|    1137|    Yes|     No|    2499.02|51504.29396|
|       2|     No|    Yes|817.1804066| 12106.1347|
|    6076|    Yes|     PT|    2413.32|38540.57271|
|    9874|     No|     No|    2391.01|50302.90956|
|       5|     No|     PT|785.6558829|38463.49588|
|       6|     No|    Yes|919.5885305|7491.558572|
|    6335|    Yes|     PT|     2343.8|51095.29393|
|       8|     No|    Yes|808.6675043|17600.45134|
|    9894|    Yes|     PT|    2288.41|52043.56905|
|    8265|    Yes|     PT|    2236.76|37113.88307|
|      11|     No|    Yes|        0.0|21871.07309|
|      12|     No|    Yes|1220.583753|13268.56222|
|    3190|    Yes|     PT|    2228.47|27438.34899|
|    1360|    Yes|     PT|    2220.97|40725.09621|
|    9979|    Yes|     PT|    2202.46|47287.25711|
+--------+-------+-------+-----------+-----------+
only showing top 15 rows

(9988

In [7]:
# Show the average balance and income by default and student.
df2.groupBy('default').avg('income').show() # average income by default type
df2.groupBy('default').avg('balance').show() # average balance for default type
df2.groupBy('student').avg('income').show() # average income by student type
df2.groupBy('student').avg('balance').show() # average balance by student

# Another way of doing it harder to make sense of
# Grouping by default and student and getting average income and balance
#df2.groupBy('default','student').avg('income','balance').show()

+-------+------------------+
|default|       avg(income)|
+-------+------------------+
|     No|33563.697893032346|
|    Yes|32089.147124231204|
+-------+------------------+

+-------+------------------+
|default|      avg(balance)|
+-------+------------------+
|     No| 804.2003095359149|
|    Yes|1747.8216032672676|
+-------+------------------+

+-------+------------------+
|student|       avg(income)|
+-------+------------------+
|     PT|40165.518941131115|
|     No|40004.832303382434|
|    Yes| 17949.22965257556|
+-------+------------------+

+-------+------------------+
|student|      avg(balance)|
+-------+------------------+
|     PT|1299.3960019779952|
|     No| 741.3479104701828|
|    Yes| 987.7499992140657|
+-------+------------------+



# Write also in markdown in a paragraph below the above explaining whether those people who defaulted have a higher or lower average income compared to those who did not default. Your answer must be based on the results from the previous bullet point.
    Based on above summary stats a person who defaults typically has a 1000 dollar on average lower income. Based on default balance aggregation a person who does default has a 943 dollar larger credit balance. 
Answer =  People with lower income and higher credit balance are the people who default. 

In [8]:
# SELECT TWO COLUMNS FOR CORRELATION PURPOSES
df3 = df2.select('balance','income')
df3.show(10)
print((df3.count(), len(df3.columns))) # yess full selection 

+-----------+-----------+
|    balance|     income|
+-----------+-----------+
|    2499.02|51504.29396|
|817.1804066| 12106.1347|
|    2413.32|38540.57271|
|    2391.01|50302.90956|
|785.6558829|38463.49588|
|919.5885305|7491.558572|
|     2343.8|51095.29393|
|808.6675043|17600.45134|
|    2288.41|52043.56905|
|    2236.76|37113.88307|
+-----------+-----------+
only showing top 10 rows

(9988, 2)


In [9]:
#Show correlation between balance and income. Write a paragraph to tell me if there is a strong 
#correlation between the two.
spark=SparkSession.builder.appName('basic_stats').getOrCreate()



In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
assembler = VectorAssembler(inputCols=df3.columns,outputCol="features")
df_new=assembler.transform(df3)
df_new.show()

+-----------+-----------+--------------------+
|    balance|     income|            features|
+-----------+-----------+--------------------+
|    2499.02|51504.29396|[2499.02,51504.29...|
|817.1804066| 12106.1347|[817.1804066,1210...|
|    2413.32|38540.57271|[2413.32,38540.57...|
|    2391.01|50302.90956|[2391.01,50302.90...|
|785.6558829|38463.49588|[785.6558829,3846...|
|919.5885305|7491.558572|[919.5885305,7491...|
|     2343.8|51095.29393|[2343.8,51095.29393]|
|808.6675043|17600.45134|[808.6675043,1760...|
|    2288.41|52043.56905|[2288.41,52043.56...|
|    2236.76|37113.88307|[2236.76,37113.88...|
|        0.0|21871.07309|   [0.0,21871.07309]|
|1220.583753|13268.56222|[1220.583753,1326...|
|    2228.47|27438.34899|[2228.47,27438.34...|
|    2220.97|40725.09621|[2220.97,40725.09...|
|    2202.46|47287.25711|[2202.46,47287.25...|
|    2187.22|42205.12305|[2187.22,42205.12...|
|    2155.29|34787.25268|[2155.29,34787.25...|
|527.5401841|17636.53962|[527.5401841,1763...|
|     2148.9|

In [12]:
from pyspark.ml.stat import Correlation
pearson_corr = Correlation.corr(df_new,'features')
pearson_corr.show(2,False) # THERE IS NOT A STRONG CORRELATION BETWEEN THESE TWO A SMALL NEGATIVE CORRELATION

+-----------------------------------------------------------------------------------------+
|pearson(features)                                                                        |
+-----------------------------------------------------------------------------------------+
|1.0                   -0.15211397584237343  
-0.15211397584237343  1.0                   |
+-----------------------------------------------------------------------------------------+



In [78]:
#Recode the student variable using One Hot Encoding.
 #must string indexer befor one hot encoding
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler
df2.describe() #

DataFrame[summary: string, RecordID: string, default: string, student: string, balance: string, income: string]

In [81]:
# This only indexes one column at a time lets get default as numeric for our logistic output
marital_indexer = StringIndexer(inputCol="student",outputCol="student_num").fit(df2)
df4 = marital_indexer.transform(df2)
df4.show()
df4.describe()

+--------+-------+-------+-----------+-----------+-----------+
|RecordID|default|student|    balance|     income|student_num|
+--------+-------+-------+-----------+-----------+-----------+
|    1137|    Yes|     No|    2499.02|51504.29396|        0.0|
|       2|     No|    Yes|817.1804066| 12106.1347|        1.0|
|    6076|    Yes|     PT|    2413.32|38540.57271|        2.0|
|    9874|     No|     No|    2391.01|50302.90956|        0.0|
|       5|     No|     PT|785.6558829|38463.49588|        2.0|
|       6|     No|    Yes|919.5885305|7491.558572|        1.0|
|    6335|    Yes|     PT|     2343.8|51095.29393|        2.0|
|       8|     No|    Yes|808.6675043|17600.45134|        1.0|
|    9894|    Yes|     PT|    2288.41|52043.56905|        2.0|
|    8265|    Yes|     PT|    2236.76|37113.88307|        2.0|
|      11|     No|    Yes|        0.0|21871.07309|        1.0|
|      12|     No|    Yes|1220.583753|13268.56222|        1.0|
|    3190|    Yes|     PT|    2228.47|27438.34899|     

DataFrame[summary: string, RecordID: string, default: string, student: string, balance: string, income: string, student_num: string]

In [86]:
# Creating a pipeline for muliple columns to be indexed if needed
# here we convert all of our categorical columns to string
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df2) for column in list(set(df2.columns)-set(['RecordID'])) ]


pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(df2).transform(df2)

df_r.show()

+--------+-------+-------+-----------+-----------+-------------+-------------+------------+-------------+
|RecordID|default|student|    balance|     income|student_index|default_index|income_index|balance_index|
+--------+-------+-------+-----------+-----------+-------------+-------------+------------+-------------+
|    1137|    Yes|     No|    2499.02|51504.29396|          0.0|          1.0|      4409.0|       1781.0|
|       2|     No|    Yes|817.1804066| 12106.1347|          1.0|          0.0|      1880.0|       7584.0|
|    6076|    Yes|     PT|    2413.32|38540.57271|          2.0|          1.0|      7772.0|       7911.0|
|    9874|     No|     No|    2391.01|50302.90956|          0.0|          0.0|      6486.0|       6524.0|
|       5|     No|     PT|785.6558829|38463.49588|          2.0|          0.0|      4620.0|       4331.0|
|       6|     No|    Yes|919.5885305|7491.558572|          1.0|          0.0|      3885.0|       7942.0|
|    6335|    Yes|     PT|     2343.8|51095.29

In [89]:
# lets assign dataframe to df4
# now lets select the appropriate columns in the data frame 
df4 = df_r
df4 = df4.select("default_index","balance","income","student_index")
df4 = df4.withColumnRenamed("default_index", "label") # rename column for ease of prediction
df4.show()

+-----+-----------+-----------+-------------+
|label|    balance|     income|student_index|
+-----+-----------+-----------+-------------+
|  1.0|    2499.02|51504.29396|          0.0|
|  0.0|817.1804066| 12106.1347|          1.0|
|  1.0|    2413.32|38540.57271|          2.0|
|  0.0|    2391.01|50302.90956|          0.0|
|  0.0|785.6558829|38463.49588|          2.0|
|  0.0|919.5885305|7491.558572|          1.0|
|  1.0|     2343.8|51095.29393|          2.0|
|  0.0|808.6675043|17600.45134|          1.0|
|  1.0|    2288.41|52043.56905|          2.0|
|  1.0|    2236.76|37113.88307|          2.0|
|  0.0|        0.0|21871.07309|          1.0|
|  0.0|1220.583753|13268.56222|          1.0|
|  1.0|    2228.47|27438.34899|          2.0|
|  1.0|    2220.97|40725.09621|          2.0|
|  1.0|    2202.46|47287.25711|          2.0|
|  1.0|    2187.22|42205.12305|          2.0|
|  1.0|    2155.29|34787.25268|          2.0|
|  0.0|527.5401841|17636.53962|          1.0|
|  1.0|     2148.9|44309.91717|   

In [90]:
# now upload appropriate stuff for one hot encoder

marital_encoder = OneHotEncoder(inputCol="student_index",outputCol="student_vector")
df5 = marital_encoder.transform(df4)
df5.describe()
df5.show()


+-----+-----------+-----------+-------------+--------------+
|label|    balance|     income|student_index|student_vector|
+-----+-----------+-----------+-------------+--------------+
|  1.0|    2499.02|51504.29396|          0.0| (2,[0],[1.0])|
|  0.0|817.1804066| 12106.1347|          1.0| (2,[1],[1.0])|
|  1.0|    2413.32|38540.57271|          2.0|     (2,[],[])|
|  0.0|    2391.01|50302.90956|          0.0| (2,[0],[1.0])|
|  0.0|785.6558829|38463.49588|          2.0|     (2,[],[])|
|  0.0|919.5885305|7491.558572|          1.0| (2,[1],[1.0])|
|  1.0|     2343.8|51095.29393|          2.0|     (2,[],[])|
|  0.0|808.6675043|17600.45134|          1.0| (2,[1],[1.0])|
|  1.0|    2288.41|52043.56905|          2.0|     (2,[],[])|
|  1.0|    2236.76|37113.88307|          2.0|     (2,[],[])|
|  0.0|        0.0|21871.07309|          1.0| (2,[1],[1.0])|
|  0.0|1220.583753|13268.56222|          1.0| (2,[1],[1.0])|
|  1.0|    2228.47|27438.34899|          2.0|     (2,[],[])|
|  1.0|    2220.97|40725

In [91]:
# Select appropriate columns
df6 = df5.select("label","balance","income","student_vector")
df6.show()

+-----+-----------+-----------+--------------+
|label|    balance|     income|student_vector|
+-----+-----------+-----------+--------------+
|  1.0|    2499.02|51504.29396| (2,[0],[1.0])|
|  0.0|817.1804066| 12106.1347| (2,[1],[1.0])|
|  1.0|    2413.32|38540.57271|     (2,[],[])|
|  0.0|    2391.01|50302.90956| (2,[0],[1.0])|
|  0.0|785.6558829|38463.49588|     (2,[],[])|
|  0.0|919.5885305|7491.558572| (2,[1],[1.0])|
|  1.0|     2343.8|51095.29393|     (2,[],[])|
|  0.0|808.6675043|17600.45134| (2,[1],[1.0])|
|  1.0|    2288.41|52043.56905|     (2,[],[])|
|  1.0|    2236.76|37113.88307|     (2,[],[])|
|  0.0|        0.0|21871.07309| (2,[1],[1.0])|
|  0.0|1220.583753|13268.56222| (2,[1],[1.0])|
|  1.0|    2228.47|27438.34899|     (2,[],[])|
|  1.0|    2220.97|40725.09621|     (2,[],[])|
|  1.0|    2202.46|47287.25711|     (2,[],[])|
|  1.0|    2187.22|42205.12305|     (2,[],[])|
|  1.0|    2155.29|34787.25268|     (2,[],[])|
|  0.0|527.5401841|17636.53962| (2,[1],[1.0])|
|  1.0|     2

In [102]:
# make label and feature columns
VecAssembler=VectorAssembler(inputCols=['balance','income', 'student_vector'],outputCol='features')
df7 = VecAssembler.transform(df6)
df7.show(20,False)

+-----+-----------+-----------+--------------+---------------------------------+
|label|balance    |income     |student_vector|features                         |
+-----+-----------+-----------+--------------+---------------------------------+
|1.0  |2499.02    |51504.29396|(2,[0],[1.0]) |[2499.02,51504.29396,1.0,0.0]    |
|0.0  |817.1804066|12106.1347 |(2,[1],[1.0]) |[817.1804066,12106.1347,0.0,1.0] |
|1.0  |2413.32    |38540.57271|(2,[],[])     |[2413.32,38540.57271,0.0,0.0]    |
|0.0  |2391.01    |50302.90956|(2,[0],[1.0]) |[2391.01,50302.90956,1.0,0.0]    |
|0.0  |785.6558829|38463.49588|(2,[],[])     |[785.6558829,38463.49588,0.0,0.0]|
|0.0  |919.5885305|7491.558572|(2,[1],[1.0]) |[919.5885305,7491.558572,0.0,1.0]|
|1.0  |2343.8     |51095.29393|(2,[],[])     |[2343.8,51095.29393,0.0,0.0]     |
|0.0  |808.6675043|17600.45134|(2,[1],[1.0]) |[808.6675043,17600.45134,0.0,1.0]|
|1.0  |2288.41    |52043.56905|(2,[],[])     |[2288.41,52043.56905,0.0,0.0]    |
|1.0  |2236.76    |37113.883

In [103]:
# Select only label and features
df8 = df7.select("label","features")
df8.show(20,False)

+-----+---------------------------------+
|label|features                         |
+-----+---------------------------------+
|1.0  |[2499.02,51504.29396,1.0,0.0]    |
|0.0  |[817.1804066,12106.1347,0.0,1.0] |
|1.0  |[2413.32,38540.57271,0.0,0.0]    |
|0.0  |[2391.01,50302.90956,1.0,0.0]    |
|0.0  |[785.6558829,38463.49588,0.0,0.0]|
|0.0  |[919.5885305,7491.558572,0.0,1.0]|
|1.0  |[2343.8,51095.29393,0.0,0.0]     |
|0.0  |[808.6675043,17600.45134,0.0,1.0]|
|1.0  |[2288.41,52043.56905,0.0,0.0]    |
|1.0  |[2236.76,37113.88307,0.0,0.0]    |
|0.0  |[0.0,21871.07309,0.0,1.0]        |
|0.0  |[1220.583753,13268.56222,0.0,1.0]|
|1.0  |[2228.47,27438.34899,0.0,0.0]    |
|1.0  |[2220.97,40725.09621,0.0,0.0]    |
|1.0  |[2202.46,47287.25711,0.0,0.0]    |
|1.0  |[2187.22,42205.12305,0.0,0.0]    |
|1.0  |[2155.29,34787.25268,0.0,0.0]    |
|0.0  |[527.5401841,17636.53962,0.0,1.0]|
|1.0  |[2148.9,44309.91717,0.0,0.0]     |
|1.0  |[2133.46,28237.88234,0.0,0.0]    |
+-----+---------------------------

In [104]:
# Make 75 train and 25 test with printing out the number of rows in each
train, test = df8.randomSplit([0.75, 0.25])
print(f"Size of train Dataset : {train.count()}" )
print(f"Size of test Dataset : {test.count()}" )

Size of train Dataset : 7563
Size of test Dataset : 2425


In [105]:
# import package needed for logistic regression
from pyspark.ml.classification import LogisticRegression
log_reg=LogisticRegression().fit(train)

In [107]:
summary=log_reg.summary
log_reg.summary.accuracy

log_reg.summary.areaUnderROC

print(log_reg.summary.precisionByLabel)

print(log_reg.summary.recallByLabel)
predictions = log_reg.transform(test)
predictions.show(10)

[0.9774557165861514, 0.7207207207207207]
[0.9957621326042378, 0.3225806451612903]
+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(4,[1],[36302.528...|[10.9583369896521...|[0.99998258806037...|       0.0|
|  0.0|(4,[1],[42284.682...|[10.9501964788627...|[0.99998244574229...|       0.0|
|  0.0|[0.0,10276.25611,...|[11.6015882796459...|[0.99999084854276...|       0.0|
|  0.0|[0.0,10593.92125,...|[10.8123115050393...|[0.99997985051150...|       0.0|
|  0.0|[0.0,13760.29846,...|[11.5968471971493...|[0.99999080505234...|       0.0|
|  0.0|[0.0,14874.80021,...|[11.5953305839143...|[0.99999079109670...|       0.0|
|  0.0|[0.0,15759.91517,...|[11.5941261200695...|[0.99999077999833...|       0.0|
|  0.0|[0.0,15986.24197,...|[11.5938181347117...|[0.99999077715830...|       0.0|
|  0.0|[0.0,1645